<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2_ContentBasedFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

##Content Based Filtering 

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-03-21 03:27:12--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-03-21 03:27:13--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-03-21 03:27:13--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133,

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import time
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
import spacy
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


### Description Based Recommender (Cosine Similarity)

In [3]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

In [4]:
product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,NaN,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...
2,Appliances Parts & Accessories,NaN,the cigar moments of pleasure,the cigar book,NaN,amazon home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,NaN,tools home improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...


In [5]:
product_df.shape

(30239, 13)

In [6]:
# Replace all NaN with an empty string
product_df = product_df.fillna('')
product_df.isnull().sum()

category           0
description        0
title              0
brand              0
feature            0
main_cat           0
date               0
price              0
asin               0
imageURLHighRes    0
dateYear           0
dateMonth          0
ori_title          0
dtype: int64

In [7]:
stop = stopwords.words('english')

In [8]:
lem = WordNetLemmatizer()
def lemma(text):
    return ' '.join(lem.lemmatize(w) for w in text.split() if w not in stop)   

In [9]:
product_df['description'] = product_df['description'].apply(lemma)

In [10]:
# Define a TF-IDF Vectorizer object and remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english', max_df = 0.9, min_df = 5)

In [11]:
# Construct the required TF-IDF matrix 
tfidf_matrix = tfidf.fit_transform(product_df['description'])

In [12]:
# Output the shape of tfidf_matrix
tfidf_matrix.shape

(30239, 9568)

Both linear_kernel and cosine_similarity methods produce the same result. We would like to apply both methods can see the performance of creating the cosine similarity matrix, and choose the faster one.

In [13]:
start = time.time()
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print("Time taken: %s seconds" % (time.time() - start))

Time taken: 13.161040306091309 seconds


In [14]:
# start = time.time()
# cosine_sim2 = cosine_similarity(tfidf_matrix, tfidf_matrix)
# print("Time taken: %s seconds" % (time.time() - start))

The performance results are very close, and the linear_kernel result will be used

In [15]:
# Reverse mapping of indices and product id
indices = pd.Series(product_df.index, index=product_df['asin'].str.lower()).drop_duplicates()

In [66]:
# Function that takes in product id as input and gives recommendations 
def description_recommender(id, cosine_sim = cosine_sim, df = product_df, indices = indices):
  # get the index of the product that matches the id
  idx = indices[id.lower()]

  # get the pairwsie similarity scores 
  # then convert it into a list of tuples as described above
  sim_scores = list(enumerate(cosine_sim[idx]))

  # sort the product based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # get the scores of the 10 most similar product. Ignore the first one because it is the input product.
  sim_scores = sim_scores[1:11]

  # get the product indices
  product_indices = [i[0] for i in sim_scores]

  # return the top 10 most similar product
  return df['asin'].iloc[product_indices].tolist(), df['ori_title'].iloc[product_indices].tolist()

In [61]:
product_df[product_df['title'].str.contains('refrigerator', case=False)].head(1)

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title,all_text,Doc,Topic,Probability
131,"Appliances Refrigerators, Freezers & Ice Makers",this compact mini cooler and warmer holds 17 l...,coldmate mr 128 mini cooler warmer deluxe mini...,coldmate,press the cold button to cool to 40 f and hot ...,appliances,2001-10-02,,B0001YH10C,['https://images-na.ssl-images-amazon.com/imag...,2001,10,Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini...,appliances refrigerators freezers ice makers t...,131,1,0.366799


In [62]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

In [67]:
#Get recommendations for Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini Refrigerator, input the product id 
asin, title = description_recommender(input_id)

In [68]:
print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin[i], title[i]))

Description Based Recommender Result for B0001YH10C, coldmate mr 128 mini cooler warmer deluxe mini refrigerator: 
1. B00ID8CLMG, Avanti FF45006W 4.3 CF Frost Free Refrigerator Freezer, White
2. B00RNAH5OY, goFridge Mini Fridge Portable Electric Cooler
3. B001H80RN4, Frigidaire 241505301 Refrigerator Door Bin Genuine Original Equipment Manufacturer (OEM) Part
4. B004NEYPYQ, Frost-Free 4.3 Cu. Ft. Refrigerator/Freezer White
5. B000JLL3BK, Pek Vino Vault Wine Preserving Refrigerator, Silver
6. B01F79MKME, Amana AMA43BK Compact Single Door Refrigerator, 4.3 cu. ft, Black
7. B00OVI6HHW, Avanti AR4456SS Counterhigh Refrigerator, 4.5 cu. ft, Black/Stainless Steel
8. B001TIYPI0, Whirlpool Part Number 2179374: Wine Rack
9. B001F7H4RY, PORTABLE COOLER WARMER MINI FRIDGE WINE BEER
10. B001775T4C, Nostalgia Electrics CRF170RETRORED Retro Series Mini Fridge, 1.7 Cubic Feet


<h3>Was trying to save the calculated cosine similarity matrix for web api use, but the matrix size is too large for local RAM or web hosting service storage and RAM. Try using another model for Web API deployment for content-based filtering. </h3>

In [21]:
# np.save('cosine_sim', cosine_sim)

In [22]:
# original_cs = np.load("cosine_sim.npy")
# original_cs

In [23]:
# get input id and title for the recommendation
# input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[1]['asin']
# input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[1]['title']

In [24]:
#Get recommendations
# asin, title = description_recommender(input_id, cosine_sim=original_cs)

In [25]:
# print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
# for i in range(0, 10):
#   print('{}. {}, {}'.format(i+1, asin[i], title[i]))

### Metadata Based Recommender (Cosine Similarity)

In [26]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')
product_df = product_df.fillna('')

In [27]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30239 entries, 0 to 30238
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         30239 non-null  object
 1   description      30239 non-null  object
 2   title            30239 non-null  object
 3   brand            30239 non-null  object
 4   feature          30239 non-null  object
 5   main_cat         30239 non-null  object
 6   date             30239 non-null  object
 7   price            30239 non-null  object
 8   asin             30239 non-null  object
 9   imageURLHighRes  30239 non-null  object
 10  dateYear         30239 non-null  int64 
 11  dateMonth        30239 non-null  int64 
 12  ori_title        30239 non-null  object
dtypes: int64(2), object(11)
memory usage: 3.0+ MB


In [28]:
# Use meta data except description and features
product_df['meta_text'] = product_df['category'] + ' ' +  product_df['title'] + ' ' +  product_df['brand'] + ' ' + product_df['main_cat'] + ' ' + product_df['price'].astype(str) + ' ' + product_df['dateYear'].astype(str) + ' ' + product_df['dateMonth'].astype(str) 
product_df['meta_text'] = product_df['meta_text'].apply(lemma)

In [29]:
# Define a TF-IDF Vectorizer object and remove all english stopwords
tfidf2 = TfidfVectorizer(stop_words='english', max_df = 0.9, min_df = 5)

In [30]:
tfidf_matrix2 = tfidf2.fit_transform(product_df['meta_text'])

In [31]:
tfidf_matrix2.shape

(30239, 3381)

In [32]:
cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)

In [33]:
indices = pd.Series(product_df.index, index=product_df['asin'].str.lower()).drop_duplicates()

In [34]:
# Function that takes in product id as input and gives recommendations 
def description_recommender(id, cosine_sim = cosine_sim2, df = product_df, indices = indices):
  # get the index of the product that matches the id
  idx = indices[id.lower()]

  # get the pairwsie similarity scores 
  # then convert it into a list of tuples as described above
  sim_scores = list(enumerate(cosine_sim[idx]))

  # sort the product based on the cosine similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # get the scores of the 10 most similar product. Ignore the first one because it is the input product.
  sim_scores = sim_scores[1:11]

  # get the product indices
  product_indices = [i[0] for i in sim_scores]

  # return the top 10 most similar product
  return df['asin'].iloc[product_indices].tolist(), df['ori_title'].iloc[product_indices].tolist()

In [35]:
product_df[product_df['title'].str.contains('refrigerator', case=False)].head(1)

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title,meta_text
131,"Appliances Refrigerators, Freezers & Ice Makers",this compact mini cooler and warmer holds 17 l...,coldmate mr 128 mini cooler warmer deluxe mini...,coldmate,press the cold button to cool to 40 f and hot ...,appliances,2001-10-02,,B0001YH10C,['https://images-na.ssl-images-amazon.com/imag...,2001,10,Coldmate MR-128 Mini Cooler/Warmer Deluxe Mini...,"Appliances Refrigerators, Freezers & Ice Maker..."


In [36]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

In [37]:
#Get recommendations
asin2, title2 = description_recommender(input_id, cosine_sim=cosine_sim2)

In [38]:
print('Description Based Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin2[i], title2[i]))

Description Based Recommender Result for B0001YH10C, coldmate mr 128 mini cooler warmer deluxe mini refrigerator: 
1. B001F7H4RY, PORTABLE COOLER WARMER MINI FRIDGE WINE BEER
2. B00YNNEC8Q, Mini Wine Cooler
3. B00ND5CWAA, Phoenix USB 5v Portable One Zip-top Can Cooler-mini Car Compact Refrigerator and Warmer
4. B00YNMUYV6, Mini Wine Cooler Refrigerator with Lock
5. B00RNAH5OY, goFridge Mini Fridge Portable Electric Cooler
6. B016K4J3U2, Honeykoko Mini USB PC Refrigerator Fridge Beverage Drink Can Cooler Warmer Heater Gadget One Can in Home Office
7. B016KQ7X8E, ThreeH New Mini Red USB Fridge Cooler Beverage Drink Cans Cooler/Warmer Refrigerator for Laptop PC Computer Red H-UF05Red
8. B0187KYRQC, Coca-Cola Mini Can Cooler
9. B00KE7FM3O, Mini USB Desktop Fridge Cooler Refrigerator
10. B005JAVC94, Mini Desktop Fridge Cooler Personal Fridge(Black)


### Topic Modeling Recommender (LDA)

In [39]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')
product_df = product_df.fillna('')

In [40]:
product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title
0,"Appliances Refrigerators, Freezers & Ice Makers",,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...
2,Appliances Parts & Accessories,,the cigar moments of pleasure,the cigar book,,amazon home,,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,,tools home improvement,2014-12-17,,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...


In [41]:
product_df['all_text'] = product_df['category'] + ' ' + product_df['description'] + ' ' + product_df['title'] + ' ' + product_df['brand'] + ' ' + product_df['feature'] +\
                         product_df['main_cat']

In [44]:
words = set(nltk.corpus.words.words())

def clear_text(doc):
  #cleaned_text = " ".join(w for w in nltk.wordpunct_tokenize(cleaned_text) if w.lower() in words or not w.isalpha())
  cleaned_text = re.sub('[^a-zA-Z0-9]', ' ', doc)
  cleaned_text = re.sub('\s\s+', ' ', cleaned_text)
  cleaned_text = cleaned_text.lower()
  return cleaned_text

In [45]:
product_df['all_text'] = product_df['all_text'].apply(clear_text)

In [46]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations and special characters

data_words = list(sent_to_words(product_df['all_text']))

In [47]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop] for doc in texts]

In [48]:
# Build the bigram 
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [49]:
lem = WordNetLemmatizer()
def lemma(texts):
    return [[lem.lemmatize(w) for w in simple_preprocess(str(doc))] for doc in texts]

In [50]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization
data_lemmatized = lemma(data_words_bigrams)
data_lemmatized[0]

['appliance',
 'ice_maker',
 'tupperware',
 'freezer',
 'square',
 'round',
 'container',
 'set',
 'tupperware',
 'pc',
 'set',
 'includes',
 'two',
 'cup_ml',
 'one',
 'cup_ml',
 'use',
 'keep',
 'sandwich',
 'filling',
 'salad',
 'leftover',
 'fresh',
 'refrigerator',
 'gently',
 'twist',
 'container',
 'pop',
 'frozen_food',
 'reheating',
 'dishwasher_safe',
 'set',
 'weight',
 'le',
 'oz',
 'appliance']

Create the two main inputs for the LDA topic model: the dictionary(id2word) and the corpus. 

In [51]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Filter out tokens that appear in only 1 documents and appear in more than 90% of the documents
id2word.filter_extremes(no_below=2, no_above=0.9)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 3), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1)]]


In [153]:
# supporting function
def compute_coherence_values(corpus, dictionary, k):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [154]:
# Iterate over possible number of topics.
topic_nums = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
coherence_scores = []

for k in topic_nums:
    coherence_score = compute_coherence_values(corpus=corpus, dictionary=id2word, k=k)
    coherence_scores.append(coherence_score)

coherence_df = pd.DataFrame()
coherence_df["topic_num"] = topic_nums
coherence_df['coherence_score'] = coherence_scores
coherence_df

Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-12:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)


KeyboardInterrupt: ignored

  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/gensim/models/ldamulticore.py", line 333, in worker_e_step
    worker_lda.do_estep(chunk)  # TODO: auto-tune alpha?
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 105, in worker
    initializer(*initargs)
  File "/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py", line 725, in do_estep
    gamma, sstats = self.inference(chunk, collect_sstats=True)
  File "/usr/local/lib/python3.7/dist-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py", line 690, in inference
    sstats[:, ids] += np.outer(expElogthetad.T, cts / phinorm)
  File "/usr/lib/pytho

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(topic_nums, coherence_df['coherence_score'])

plt.title("Coherence Scores vs Topic Numbers for LDA Model")
plt.xlabel("Topic Numbers")
plt.ylabel("Coherence Scores")
plt.show()

In [52]:
# Build LDA model
k=10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=k, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [53]:
lda_model.print_topics()

[(0,
  '0.044*"range_hood" + 0.031*"stainless_steel" + 0.025*"filter" + 0.018*"speed" + 0.015*"control" + 0.014*"inch" + 0.013*"cfm" + 0.012*"hood" + 0.010*"fan" + 0.010*"air"'),
 (1,
  '0.045*"refrigerator" + 0.027*"temperature" + 0.026*"ice_maker" + 0.023*"door" + 0.022*"ice" + 0.020*"wr" + 0.019*"freezer" + 0.011*"shelf" + 0.010*"humidity" + 0.010*"cu_ft"'),
 (2,
  '0.167*"part" + 0.053*"accessory" + 0.047*"oem" + 0.039*"fit" + 0.036*"whirlpool" + 0.032*"ge" + 0.031*"number" + 0.031*"improvement" + 0.025*"authorized" + 0.025*"brand_models"'),
 (3,
  '0.167*"part" + 0.085*"replacement" + 0.069*"accessory" + 0.059*"genuine" + 0.032*"frigidaire" + 0.026*"model_number" + 0.024*"name" + 0.024*"dryer" + 0.023*"following_item" + 0.022*"whirlpool"'),
 (4,
  '0.012*"easy" + 0.011*"use" + 0.007*"one" + 0.007*"home" + 0.006*"clean" + 0.006*"time" + 0.006*"limited_time" + 0.006*"add" + 0.006*"logo" + 0.006*"make"'),
 (5,
  '0.072*"filter" + 0.072*"water" + 0.029*"refrigerator" + 0.029*"replacem

In [70]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('LDA model Coherence Score: ', coherence_lda)

LDA model Coherence Score:  0.5709976851379508


In [83]:
topic_num = []
probability = []

for n in range(len(product_df)):
  # predict the topic number
  get_document_topics = lda_model.get_document_topics(corpus[n])

  # sort by each topic numbers' probability
  sorted_doc_topics = (sorted(get_document_topics, key = lambda x: x[1], reverse = True)) 

  # get the top topic number
  topic_num.append(sorted_doc_topics[0][0]) 

   # get the top topic number probability
  probability.append(sorted_doc_topics[0][1])

product_df['topic_num'] = topic_num
product_df['probability'] = probability


In [84]:
product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title,all_text,Doc,Topic,Probability,topic_num,probability
0,"Appliances Refrigerators, Freezers & Ice Makers",,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...,appliances refrigerators freezers ice makers t...,0,4,0.838047,4,0.838050
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...,appliances refrigerators freezers ice makers 2...,1,1,0.436292,1,0.436292
2,Appliances Parts & Accessories,,the cigar moments of pleasure,the cigar book,,amazon home,,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure,appliances parts accessories the cigar moments...,2,7,0.673443,7,0.673443
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,,tools home improvement,2014-12-17,,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene,appliances parts accessories multi purpost des...,3,2,0.346032,2,0.346007
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,appliances parts accessories range parts acces...,4,4,0.351157,4,0.351166


In [147]:
def topic_modeling_recommender(id, df):
    id = id.lower()

    # get the input product topic number
    topic_num = df[df['asin'].str.lower() == id]['topic_num'].item()

    # remove the input product from the recommendation data
    exclude_input_df = df.copy()
    exclude_input_df = exclude_input_df[exclude_input_df['asin'].str.lower() != id]

    # get the top 10 Probability product for the matching topic number
    output_df = exclude_input_df[exclude_input_df['topic_num'] == topic_num].sort_values('Probability', ascending=False).head(10)

    # get the product indices
    product_indices = output_df.index.tolist()

    # return the top 10 most similar product
    return df['asin'].iloc[product_indices].tolist(), df['ori_title'].iloc[product_indices].tolist()

In [148]:
# get input id and title for the recommendation
input_id = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['asin']
input_title = product_df[product_df['title'].str.contains('refrigerator', case=False)].iloc[0]['title']

In [151]:
#Get recommendations
asin3, title3 = topic_modeling_recommender('B008XCK6SI', product_df)

In [152]:
print('Topic Modeling Recommender Result for {}, {}: '.format(input_id, input_title))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin3[i], title3[i]))

Topic Modeling Recommender Result for B0001YH10C, coldmate mr 128 mini cooler warmer deluxe mini refrigerator: 
1. B001ASTT0S, GE GSCS3PGXSS 22.7 Cu. Ft. Stainless Steel Counter Depth Side-By-Side Refrigerator
2. B0076CVOW8, GE GTS18CBDBB 18.1 Cu. Ft. Black Top Freezer Refrigerator
3. B001BXC2LK, Summit : FF1325SS 10.4 cu. ft. Counter-Depth Top-Freezer Refrigerator - Stainless Steel Doors
4. B0018S1V88, GE GSL25JFXLB 25 cu. Ft. Side by Side Refrigerator with Water and Ice Dispenser - CleanSteel
5. B00EZ2EWLM, GE GZS23HGEWW 22.7 Cu. Ft. White Counter Depth Side-By-Side Refrigerator
6. B00CXNLBBO, Amana ASD2575BRW 25.5 Cu. Ft. White Side-By-Side Refrigerator - Energy Star
7. B0074WFW28, GE GTH17DBDWW 16.5 Cu. Ft. White Top Freezer Refrigerator - Energy Star
8. B003V1CP26, GE Profile: PGCS1PJZSS 20.6 cu. ft. Counter-Depth French Door Refrigerator with 4 Adjustable Glass Shelves, Armoire Style Double Drawer Freezer System and External Water Dispenser
9. B001ASOVMO, GE PSDS5YGXSS Profile 24

In [ ]:
product_df.to_csv("Content_based_LDA_output.csv", index=False)